# 函数式编程风格和流

## 求素数

一个没有优化的朴素算法。

In [121]:
function isPrime(n) {
  if (n < 2) {
    return false;
  }

  // 最简单的优化是对n开个根号
  for (let i = 2; i < n; i++) {
    if (n % i === 0) {
      return false;
    }
  }
  return true;
}

In [122]:
isPrime(2)

true

In [123]:
isPrime(11)

true

In [124]:
isPrime(2**13 - 1)

true

In [125]:
isPrime(2**11-1)

false

先用命令式风格实现两片代码。

## 计算1000内所有素数的和

In [126]:
let s = 0;
const n = 1000;
for (let i = 1; i <= n; i++) {
  if (isPrime(i)) {
    s += i;
  }
}
s

76127

## 找出比1000大的3个素数

In [127]:
const list = [];
let start = 1000;
while (list.length < 3) {
  if (isPrime(start)) {
    list.push(start);
  }
  start++;
}

list

[ 1009, 1013, 1019 ]

现在看函数式编程如何表达上述问题。

## 函数式工具

In [128]:
function filter(f, ls) {
 return ls.filter(f)
}

function range(n) {
  return [...Array(n).keys()]
}

function sum(ls) {
  return ls.reduce((a, b) => a + b, 0)
}

## 计算1000内所有素数和

In [129]:
sum(filter(isPrime, range(1000)))

76127

In [130]:
sum(filter(isPrime, range(2000)))

277050

如果要应用于更多的数字呢？按惯例我们需要定义一个函数来封装上述逻辑。

## 再来一遍

In [131]:
const filter = f => ls => ls.filter(f)
const filterPrime = filter(isPrime)
const sumOfPrime = x => sum(filterPrime(x))

sumOfPrime(range(1000))
sumOfPrime(range(2000))

277050

## compose

使用函数组合(compose)让上述意图更明确。

In [132]:
const filter = f => ls => ls.filter(f)
const compose = (f, g) => x => f(g(x))

const sumOfPrime = compose(  // 函数的组合
  sum,                 // 2. 求和
  filter(isPrime),     // 1. 过滤出素数
)

sumOfPrime(range(1000))

76127

compose是右结合的，所以上述执行流程是：先过滤再求和。

想像成数据从右边流入，从左边流出。

```
output <<-- sum <<-- filter(isPrimce) <<-- input
```

注意到compose配合curry形式的filter使用时， 数据(data) 消失了， 这叫pointfree, 这是另一个故事。

我们还是先看第二个问题。

## 找出比1000大的3个素数

如何使用函数式风格来编写？

计划先实现较一个拙略的版本，像这样：

In [133]:
// take(3, filter(isPrime, range(1000, 2000)))

扩展之前的range，以支持区间。

In [134]:
function range([start, end]) {
  return Array(end - start).fill(0).map((_, i) => start + i)
}

In [135]:
range([1, 5])

[ 1, 2, 3, 4 ]

take用于取列表的头部元素。

In [136]:
function take(n, list) {
  return list.slice(0, n)
}

In [137]:
take(3, filter(isPrime, range([1000, 2000])))

[ 1009, 1013, 1019 ]

## 问题

上述实现有两个问题：

- 如果要求>1000的100个素数，range的右区间不知道定多少好;
- 虽然结果只取3个数字，但实际执行时，把 1000~2000 内的所有素数都给找出来了，进行了不必要的计算。

那有什么办法即能用函数式风格表达，又能够按需执行呢？ 

即要又要是非常重要的。


想来，我们需要一种 “链”一般的结构，链上的每个元素是"lazy"的，这种结构就是流（Stream)

## 自然数

我们从自然数开始探索这种结构，在基于集合论的现代数学中，它从0开始。

In [138]:
function N(n = 0) {
  return { value: n, next: () => N(n + 1) }
}

In [139]:
N()

{ value: 0, next: [Function: next] }

In [140]:
let o = N()
for (let i = 0; i < 10; i++) {
  console.log(o.value)
  o = o.next()
}

0
1
2
3
4
5
6
7
8
9


{ value: 10, next: [Function: next] }

注意到它是immutable和pure的。

### 改写一下

注意到 构造器(N) 和 next其实是一个东西，换一种视角。

In [141]:
const N = (n = 0) => {
  return { value: n, next: () => N(n + 1) }
}

let cur = N
for (let i = 0; i < 10; i++) {
  const iter = cur()  // 实例化元素， 即取次前调用。
    
  console.log(iter.value)
    
  cur = iter.next // 链式操作
}

0
1
2
3
4
5
6
7
8
9


[Function: next]

## Stream 

上述结构用类型描述就是

```ts
type Stream<T> = () => StreamItem<T>
type StreamItem<T> = {
  value: T
  next: Stream<T>
}
```

接下来我们在此结构上定义一些操作。

### take

In [142]:
// Number, Stream<T> -> [T]

function take(n, st) {
  const list = []
  let cur = st
  for (let i = 0; i < n && cur; i++) {
    const iter = cur()
    list.push(iter.value)
    cur = iter.next
  }
  return list
}

In [143]:
take(7, N)

[ 0, 1, 2, 3, 4, 5, 6 ]

### 过滤

In [151]:
function filter(f, st) {
  return () => {
    const iter = st()
    const next = iter.next ? filter(f, iter.next) : undefined
    if (f(iter.value)) {
      return { value: iter.value, next }
    }
    return next ? next() : undefined
  }
}

## 取出前15个偶数

In [145]:
const N = (n = 0) => {
  return { value: n, next: () => N(n + 1) }
}

take(15, filter(v => v % 2 === 0, N))

[ 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28 ]

有种时间被折叠的感觉， 现在我们看看prime。

### 取出前10个素数

In [146]:
take(10, filter(isPrime, N))

[ 2, 3, 5, 7, 11, 13, 17, 19, 23, 29 ]

## 取出大于1000的3个素数

In [147]:
take(3, filter(v => v > 1000 && isPrime(v), N))

[ 1009, 1013, 1019 ]

如何求1000到2000内的所有素数

## Range


需要一个表示Range的流。

In [148]:
function Range([a, b]) {
  const iter = value => () => {
    const next = value + 1 < b ? iter(value + 1) : undefined
    return { value, next }
  }
  return iter(a)
}

In [150]:
take(10000, filter(isPrime, Range([1000, 2000])))

[ 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 1223, 1229, 1231, 1237, 1249, 1259, 1277, 1279, 1283, 1289, 1291, 1297, 1301, 1303, 1307, 1319, 1321, 1327, 1361, 1367, 1373, 1381, 1399, 1409, 1423, 1427, 1429, 1433, 1439, 1447, 1451, 1453, 1459, 1471, 1481, 1483, 1487, 1489, 1493, 1499, 1511, 1523, 1531, 1543, 1549, 1553, 1559, 1567, 1571, 1579, 1583, 1597, 1601, 1607, 1609, 1613, 1619, 1621, 1627, 1637, 1657, 1663, 1667, 1669, 1693, 1697, 1699, 1709, 1721, 1723, 1733, 1741, 1747, 1753, 1759, 1777, 1783, 1787, 1789, 1801, 1811, 1823, 1831, 1847, 1861, 1867, 1871, 1873, 1877, 1879, 1889, 1901, 1907, 1913, 1931, 1933, 1949, 1951, 1973, 1979, 1987, 1993, 1997, 1999 ]

## 映射（map)

实现流版本的map

In [152]:
function map(f, st) {
  return () => {
    const iter = st()
    const value = f(iter.value)
    const next = iter.next ? map(f, iter.next) : undefined
    return { value, next }
  }
}

In [155]:
// 偶数
const _2N = map(x => x * 2, N)

take(10, _2N)

[ 0, 2, 4, 6, 8, 10, 12, 14, 16, 18 ]

In [ ]:
// 